I downloaded daily summary data from the Oklahoma Mesonet for the Norman sites from 1994 to now.  Our first step is to load this data in using pandas (which you learned about this morning).

In [1]:
# We need to import our libraries.  Run this cell once to bring in all the correct libraries
%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.metrics import *
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('NormanMesonet.csv')

In [3]:
# check which data columns we have
# the names are documented at http://www.mesonet.org/index.php/site/about/daily_summaries
print(data.columns)

Index(['YEAR', 'MONTH', 'DAY', 'STID', 'TMAX', 'TMIN', 'TAVG', 'DMAX', 'DMIN',
       'DAVG', 'HMAX', 'HMIN', 'HAVG', 'VDEF', '9AVG', 'HDEG', 'CDEG', 'HTMX',
       'WCMN', 'PDIR', 'PDFQ', 'SDIR', 'SDFQ', 'WSMX', 'WSMN', 'WSPD', 'WDEV',
       'WMAX', '2MAX', '2MIN', '2AVG', '2DEV', 'PMAX', 'PMIN', 'PAVG', 'MSLP',
       'AMAX', 'ATOT', 'RAIN', 'RNUM', 'RMAX', 'SMAX', 'SMIN', 'SAVG', 'BMAX',
       'BMIN', 'BAVG', 'S5MX', 'S5MN', 'S5AV', 'B5MX', 'B5MN', 'B5AV', 'S25X',
       'S25N', 'S25AV', 'S3MX', 'S3MN', 'S3AV', 'S60X', 'S60N', 'S60AV',
       'B5MX.1'],
      dtype='object')


In [4]:
# we need to cleanup the data a bit since we asked for all data from both norman stations
# and one of them is only ever active at once
# let's remove all the rows where there is a -999 in the TMAX variable
cleaned_data = data[data['TMAX'] > -900]
print(cleaned_data.shape)

(8311, 63)


In [5]:
# now let's break our data into training, testing, and validation
# there are lots of ways to do this and we are picking the fastest for the purposes of time
# just breaking it up by year.
testing = cleaned_data[cleaned_data['YEAR'] == 2019]
validation = cleaned_data[cleaned_data['YEAR'] == 2018]
training = cleaned_data[cleaned_data['YEAR'] < 2018]

In [6]:
# creating multiple different labels to play with for training
def get_warmer_than_constant_labels(my_data, target_max_temp):
    """
    Return 1 if the max temperature is greater than the specified max temp and 0 otherwise
    """
    num_labels = my_data.shape[0]
    my_labels = np.zeros((num_labels,))

    # loop through each day and compare to the specified target
    example = 0
    for ind in my_data.index:
        today_max = float(my_data['TMAX'][ind])

        if (today_max >= target_max_temp):
            my_labels[example] = 1
        example +=1 
    return my_labels

def get_warmer_than_comparison(my_data):
    """
    Return 1 if the max temperature today is greater than the max temp yesterday
    """
    num_labels = my_data.shape[0]
    my_labels = np.zeros((num_labels,))

    # now loop through each day and see if the max is higher than the day before
    yesterday_max = -10000
    example = 0
    for ind in my_data.index:
        today_max = float(my_data['TMAX'][ind])

        if (today_max >= yesterday_max):
            my_labels[example] = 1
        example +=1 
        yesterday_max = today_max
    return my_labels


In [7]:
# now compute the labels for the training, validation, and testing data
# two different approaches, just uncomment the one we want to use
# training_labels = get_warmer_than_constant_labels(training, 70)
# testing_labels = get_warmer_than_constant_labels(testing, 70)
# validation_labels = get_warmer_than_constant_labels(validation, 70)

training_labels = get_warmer_than_comparison(training)
testing_labels = get_warmer_than_comparison(testing)
validation_labels = get_warmer_than_comparison(validation)

In [8]:
# now we just need to setup our tree
tree = DecisionTreeClassifier(max_depth=10)
columns = ['TMIN', 'TAVG', 'DMAX', 'DMIN', 'DAVG', 'HMAX', 'HMIN', 'HAVG', 'VDEF', '9AVG', 
           'HDEG', 'CDEG', 'HTMX', 'WCMN', 'PDIR', 'PDFQ', 'SDIR', 'SDFQ', 'WSMX', 'WSMN', 
           'WSPD', 'WDEV','WMAX', '2MAX', '2MIN', '2AVG', '2DEV', 'PMAX', 'PMIN', 'PAVG', 'MSLP',
           'AMAX', 'ATOT', 'RAIN', 'RNUM', 'RMAX', 'SMAX', 'SMIN', 'SAVG', 'BMAX','BMIN', 
           'BAVG', 'S5MX', 'S5MN', 'S5AV', 'B5MX', 'B5MN', 'B5AV', 'S25X','S25N', 'S25AV', 
           'S3MX', 'S3MN', 'S3AV', 'S60X', 'S60N', 'S60AV']
tree.fit(training[columns].values, training_labels)
             

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [9]:
# let's see how well it did
predictions = tree.predict(testing[columns].values)
auc_score = roc_auc_score(testing_labels, predictions)

print("AUC: %f" % auc_score)
acc = accuracy_score(testing_labels, predictions)
print("Accuracy %f" % acc)

AUC: 0.664548
Accuracy 0.674931


In [10]:
# can we improve using a forest?
rf = RandomForestClassifier(n_estimators=100)
rf.fit(training[columns].values, training_labels)

gbrt = GradientBoostingClassifier(n_estimators=100)
gbrt.fit(training[columns].values, training_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [11]:
# let's see how well they did
rf_predictions = rf.predict(testing[columns].values)
gbrt_predictions = gbrt.predict(testing[columns].values)

auc_score = roc_auc_score(testing_labels, rf_predictions)
print("RF AUC: %f" % auc_score)
acc = accuracy_score(testing_labels, rf_predictions)
print("RF Accuracy %f" % acc)

auc_score = roc_auc_score(testing_labels, gbrt_predictions)
print("GBRT AUC: %f" % auc_score)
acc = accuracy_score(testing_labels, gbrt_predictions)
print("GBRT Accuracy %f" % acc)

RF AUC: 0.761194
RF Accuracy 0.771350
GBRT AUC: 0.778515
GBRT Accuracy 0.787879


In [12]:
# creating a real-valued label
def get_temp_difference(my_data):
    """
    Return the difference in temperature for the day (max - min)
    """
    num_labels = my_data.shape[0]
    my_labels = np.zeros((num_labels,))

    # now loop through each day and compute the difference
    example = 0
    for ind in my_data.index:
        today_max = float(my_data['TMAX'][ind])
        today_min = float(my_data['TMIN'][ind])

        my_labels[example] = today_max - today_min
        example +=1 
    return my_labels


In [13]:
# let's setup labels for a real-valued prediction task
regression_training_labels = get_temp_difference(training)
regression_testing_labels = get_temp_difference(testing)
regression_validation_labels = get_temp_difference(validation)         

In [14]:
# and create the real-valued tree
regression_tree = DecisionTreeRegressor(max_depth=10)
regression_cols = ['DMAX', 'DMIN', 'DAVG', 'HMAX', 'HMIN', 'HAVG', 'VDEF', '9AVG', 
           'HDEG', 'CDEG', 'HTMX', 'WCMN', 'PDIR', 'PDFQ', 'SDIR', 'SDFQ', 'WSMX', 'WSMN', 
           'WSPD', 'WDEV','WMAX', '2MAX', '2MIN', '2AVG', '2DEV', 'PMAX', 'PMIN', 'PAVG', 'MSLP',
           'AMAX', 'ATOT', 'RAIN', 'RNUM', 'RMAX', 'SMAX', 'SMIN', 'SAVG', 'BMAX','BMIN', 
           'BAVG', 'S5MX', 'S5MN', 'S5AV', 'B5MX', 'B5MN', 'B5AV', 'S25X','S25N', 'S25AV', 
           'S3MX', 'S3MN', 'S3AV', 'S60X', 'S60N', 'S60AV']
regression_tree.fit(training[regression_cols].values, regression_training_labels)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [15]:
# let's see how well it did
predictions = regression_tree.predict(testing[regression_cols].values)
mse = mean_squared_error(regression_testing_labels, predictions)
print("MSE: %f" % mse)
mae = mean_absolute_error(regression_testing_labels, predictions)
print("MAE %f" % mae)

MSE: 29.349370
MAE 3.920198


In [16]:
# and try real-valued forests for comparison
regression_rf = RandomForestRegressor(n_estimators=100)
regression_rf.fit(training[regression_cols].values, regression_training_labels)

regression_gbrt = GradientBoostingRegressor(n_estimators=100)
regression_gbrt.fit(training[regression_cols].values, regression_training_labels)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [17]:
# let's see how well they did
regression_rf_predictions = regression_rf.predict(testing[regression_cols].values)
regression_gbrt_predictions = regression_gbrt.predict(testing[regression_cols].values)

mse = mean_squared_error(regression_testing_labels, regression_rf_predictions)
print("RF MSE: %f" % mse)
mae = mean_absolute_error(regression_testing_labels, regression_gbrt_predictions)
print("RF MAE %f" % mae)
mse = mean_squared_error(regression_testing_labels, regression_gbrt_predictions)
print("GBRT MSE: %f" % mse)
mae = mean_absolute_error(regression_testing_labels, regression_gbrt_predictions)
print("GBRT MAE %f" % mae)

RF MSE: 14.408144
RF MAE 2.668732
GBRT MSE: 13.115370
GBRT MAE 2.668732
